In [56]:
%%writefile func_rpdiamonds_csv.py
'''
parametrs = {'ШТУК':'ШТУК','ОГРАНКА':'ОГРАНКА', 'КАРАТ':'КАРАТ', 'ПО': 'ПО', 'Ц':'Ц', 'Д':'Д'}
rusian price of diamond v14-10-21 
def rpdiamonds({'id':1,'ОГРАНКА':['Кр-57А'], 'Ц':[3], 'Д':['7а'], 'ПО': 'A','ШТУК':[1],'КАРАТ':0.35})
'''
#
#import numpy as np
import pandas as pd
#import re
#import os
from rusprice.dictdiamonds2 import dcol_dia, dcla_dia, dpo_dia, dogr_dia_discount

parametrs = {'ШТУК':'ШТУК','ОГРАНКА':'ОГРАНКА', 'КАРАТ':'КАРАТ', 'ПО': 'ПО', 'Ц':'Ц', 'Д':'Д'}

#init globals 1
#carat_to_grp = pd.read_excel("rusprice/CARtoKVG.xlsx", sheet_name= 3).iloc[:,0:3]
carat_to_grp = pd.read_csv("rusprice/CARtoKVG.csv").drop(columns = 'Unnamed: 0')
carat_to_grp.columns = ['grp','caratmin','caratmax']


#init globals 2
#KVG57ct = pd.read_excel('rusprice/KVG57ct.xlsx')
KVG57ct = pd.read_csv('rusprice/KVG57ct.csv')
##ogrdiscount_f = pd.read_excel('rusprice/ogrdiscount_f.xlsx')
#ogrdiscount = pd.read_excel('rusprice/ogrdiscount_f.xlsx')
ogrdiscount = pd.read_csv('rusprice/ogrdiscount_f.csv')
ogrdiscount.rename(columns={'Ogr':'ogrdisc'},inplace =True)
#PRD = pd.read_excel('rusprice/PR2019_270921.xlsx').set_index('Код')
PRD = pd.read_csv('rusprice/PR2019_270921.csv').set_index('Код')
#pd.read_excel('rusprice/PR2019.xlsx').set_index('ID').columns

PRD.columns = ['OGR', 'GRUP', 'KVG', 'PO', 'Col', 'Cla', 'Pst', 'Carat', 'Pr2018',
       'Pr2019']
PRD[['Pr2018','Pr2019']] = PRD[['Pr2018','Pr2019']].astype('float64')
PRD.rename(columns={'OGR':'ogrbasic','GRUP':'Grp', 'PO':'Po','Pst':'ШТУК','Carat':"КАРАТ"}, inplace=True)


class DataError(Exception):
    pass

def checkdata(df):
    if not set(parametrs.keys()) <= set(df.columns): 
        raise DataError('проверьте состояние параметров ОГРЦДПОШК',df.columns)
    ch ={}
    ch['col'] = df.Ц.apply(lambda x: x in dcol_dia).unique()
    ch['cla'] = df.Д.apply(lambda x: x in dcla_dia).unique()
    ch['po']  = df.ПО.apply(lambda x: x in dpo_dia).unique()
    ch['ogr'] = df.ОГРАНКА.apply(lambda x: x in dogr_dia_discount).unique()
    ch['shtuk'] = [True]
    ch['carat'] = [True]
    status = ''
    err = 1
    for i in ch.keys():
        status += ' {} - {} ;'.format(i,(ch[i][0] and len(ch[i]) == 1))
        err*=(ch[i][0] and len(ch[i]) == 1)
    return err, status

def dogr_dia_basic(freeogr):
    real_ogr = dogr_dia_discount[freeogr]
    return real_ogr if real_ogr == 'Кр-57' or real_ogr == 'Кр-17' else 'Кр-57'

def grp_dia(pcs, carat):
    global  carat_to_grp
    acarat = round(carat/pcs,5)+0.00001
    res1 = carat_to_grp[round(carat_to_grp.caratmin,4) < acarat].grp.index.max()
    res2 = carat_to_grp[round(carat_to_grp.caratmax,4) > acarat].grp.index.min()
    return carat_to_grp.iloc[res1,0] if res2 == res1 else 'round error grp file'
def discont_po (po):
    dpo_dia = {'A': 'А', 'a': 'А', 'А': 'А', 'а': 'А', 'б': 'Б', 'Б': 'Б', 'В': 'В', 'в': 'В', 'В': 'В', 'г': 'Г', 'Г': 'Г' }
    dic_discont_po = {'А':1, 'Б':0.85, 'В':0.80, 'Г':0.75}
    return round(dic_discont_po[dpo_dia[po]],2)
def fillgrp(grps,grpe):
    global KVG57ct 
    l=list()
    def correctgrp(g):
        return g if g!='4.0-3.3' else '4-3.4'
    grps = correctgrp(grps)
    grpe = correctgrp(grpe)
    for i in range(KVG57ct[KVG57ct.Grp == grps].index[0],KVG57ct[KVG57ct.Grp == grpe].index[0]+1):
        l.append(KVG57ct.iloc[i,0])
    return l

def rpdiamonds(df_original):
    '''DataFrame -> DataFrame with prpr prcost'''
    df = df_original.copy()
    try:
        status = checkdata(df)
        if not status[0]:
            raise DataError(status[1])
    except DataError as e:
        print(e)
    #обработка ошибок
    print(df[~df.ОГРАНКА.apply(lambda x: x in dogr_dia_discount)].ОГРАНКА.unique())
    print(df[~df.ПО.apply(lambda x: x in dpo_dia)].ПО.unique())
    print(df[~df.Ц.apply(lambda x: x in dcol_dia)].Ц.unique())
    print(df[~df.Д.apply(lambda x: x in dcla_dia)].Д.unique())
    #a.loc[df.ОГРАНКА == 'Р-57','ОГРАНКА']
    df['СРКАРАТ'] = df.КАРАТ/df.ШТУК
    #df.loc[df.СРКАРАТ > 0.3000,'СРКАРАТ'].sort_values().unique()
    print(df.СРКАРАТ.min(),' - ', end = '')#sort_values().unique()
    print(df.СРКАРАТ.max())
    #print(df.loc[a.СРКАРАТ > 2,['id7c','vstavki','vs_dict','gem','ШТУК','ОГРАНКА','КАРАТ','СРКАРАТ','ЦД','ПО','Ц','Д']].sort_values('СРКАРАТ',ascending = False))
    diamonds = df
    diamonds['Col'] = diamonds.Ц.map(dcol_dia)
    diamonds['Cla'] = diamonds.Д.map(dcla_dia)
    diamonds['Po'] = diamonds.ПО.map(dpo_dia)
    diamonds['ogrdisc']= diamonds.ОГРАНКА.map(dogr_dia_discount)
    diamonds['Grp'] = diamonds[['ШТУК','КАРАТ']].apply(lambda x: grp_dia(x.ШТУК,x.КАРАТ), axis=1)
    diamonds['discont_po'] = diamonds.Po.map(discont_po)
    diamonds['ogrbasic'] = diamonds.ОГРАНКА.apply(lambda x: dogr_dia_basic(x))
    res0 = pd.merge(diamonds,PRD[['ogrbasic','Grp','Col','Cla','Pr2019']], how='left',left_on=['ogrbasic','Grp','Col','Cla'], right_on=['ogrbasic','Grp','Col','Cla'])
    res=pd.merge(res0,ogrdiscount[['ogrdisc','Grp','ogrdiscount']], how = 'left', left_on = ['ogrdisc','Grp'], right_on =  ['ogrdisc','Grp'])
    res['prpr'] = round(res.Pr2019*res.ogrdiscount*res.discont_po+0.0000001,0)
    res['prcost'] = round(res.prpr*res.КАРАТ,2)
    print('не сработал прейскурант на этих строках:\n  ', res.loc[pd.isna(res.prpr),['ШТУК',
       'ОГРАНКА', 'КАРАТ', 'ПО', 'Ц', 'Д', 'СРКАРАТ', 'Col',
       'Cla', 'Po', 'ogrdisc', 'Grp', 'discont_po', 'ogrbasic', 'Pr2019',
       'ogrdiscount', 'prpr', 'prcost']])
    return res

Writing func_rpdiamonds_csv.py


# tests

In [ ]:
'''
parametrs = {'ШТУК':'ШТУК','ОГРАНКА':'ОГРАНКА', 'КАРАТ':'КАРАТ', 'ПО': 'ПО', 'Ц':'Ц', 'Д':'Д'}
rusian price of diamond vxls14-10-21 
def rpdiamonds({'id':1,'ОГРАНКА':['Кр-57А'], 'Ц':[3], 'Д':['7а'], 'ПО': 'A','ШТУК':[1],'КАРАТ':0.35})
'''
#
#import numpy as np
import pandas as pd
#import re
#import os
from rusprice.dictdiamonds2 import dcol_dia, dcla_dia, dpo_dia, dogr_dia_discount

parametrs = {'ШТУК':'ШТУК','ОГРАНКА':'ОГРАНКА', 'КАРАТ':'КАРАТ', 'ПО': 'ПО', 'Ц':'Ц', 'Д':'Д'}

#init globals 1
carat_to_grp = pd.read_excel("rusprice/CARtoKVG.xlsx", sheet_name= 3).iloc[:,0:3]
carat_to_grp.columns = ['grp','caratmin','caratmax']

#init globals 2
KVG57ct = pd.read_excel('rusprice/KVG57ct.xlsx')
ogrdiscount_f = pd.read_excel('rusprice/ogrdiscount_f.xlsx')
ogrdiscount = pd.read_excel('rusprice/ogrdiscount_f.xlsx')
ogrdiscount.rename(columns={'Ogr':'ogrdisc'},inplace =True)
PRD = pd.read_excel('rusprice/PR2019_270921.xlsx').set_index('Код')
pd.read_excel('rusprice/PR2019.xlsx').set_index('ID').columns

PRD.columns = ['OGR', 'GRUP', 'KVG', 'PO', 'Col', 'Cla', 'Pst', 'Carat', 'Pr2018',
       'Pr2019']
PRD[['Pr2018','Pr2019']] = PRD[['Pr2018','Pr2019']].astype('float64')
PRD.rename(columns={'OGR':'ogrbasic','GRUP':'Grp', 'PO':'Po','Pst':'ШТУК','Carat':"КАРАТ"}, inplace=True)


class DataError(Exception):
    pass

def checkdata(df):
    if not set(parametrs.keys()) <= set(df.columns): 
        raise DataError('проверьте состояние параметров ОГРЦДПОШК',df.columns)
    ch ={}
    ch['col'] = df.Ц.apply(lambda x: x in dcol_dia).unique()
    ch['cla'] = df.Д.apply(lambda x: x in dcla_dia).unique()
    ch['po']  = df.ПО.apply(lambda x: x in dpo_dia).unique()
    ch['ogr'] = df.ОГРАНКА.apply(lambda x: x in dogr_dia_discount).unique()
    ch['shtuk'] = [True]
    ch['carat'] = [True]
    status = ''
    err = 1
    for i in ch.keys():
        status += ' {} - {} ;'.format(i,(ch[i][0] and len(ch[i]) == 1))
        err*=(ch[i][0] and len(ch[i]) == 1)
    return err, status

def dogr_dia_basic(freeogr):
    real_ogr = dogr_dia_discount[freeogr]
    return real_ogr if real_ogr == 'Кр-57' or real_ogr == 'Кр-17' else 'Кр-57'

def grp_dia(pcs, carat):
    global  carat_to_grp
    acarat = round(carat/pcs,5)+0.00001
    res1 = carat_to_grp[round(carat_to_grp.caratmin,4) < acarat].grp.index.max()
    res2 = carat_to_grp[round(carat_to_grp.caratmax,4) > acarat].grp.index.min()
    return carat_to_grp.iloc[res1,0] if res2 == res1 else 'round error grp file'
def discont_po (po):
    dpo_dia = {'A': 'А', 'a': 'А', 'А': 'А', 'а': 'А', 'б': 'Б', 'Б': 'Б', 'В': 'В', 'в': 'В', 'В': 'В', 'г': 'Г', 'Г': 'Г' }
    dic_discont_po = {'А':1, 'Б':0.85, 'В':0.80, 'Г':0.75}
    return round(dic_discont_po[dpo_dia[po]],2)
def fillgrp(grps,grpe):
    global KVG57ct 
    l=list()
    def correctgrp(g):
        return g if g!='4.0-3.3' else '4-3.4'
    grps = correctgrp(grps)
    grpe = correctgrp(grpe)
    for i in range(KVG57ct[KVG57ct.Grp == grps].index[0],KVG57ct[KVG57ct.Grp == grpe].index[0]+1):
        l.append(KVG57ct.iloc[i,0])
    return l

def rpdiamonds(df_original):
    '''DataFrame -> DataFrame with prpr prcost'''
    df = df_original.copy()
    try:
        status = checkdata(df)
        if not status[0]:
            raise DataError(status[1])
    except DataError as e:
        print(e)
    #обработка ошибок
    print(df[~df.ОГРАНКА.apply(lambda x: x in dogr_dia_discount)].ОГРАНКА.unique())
    print(df[~df.ПО.apply(lambda x: x in dpo_dia)].ПО.unique())
    print(df[~df.Ц.apply(lambda x: x in dcol_dia)].Ц.unique())
    print(df[~df.Д.apply(lambda x: x in dcla_dia)].Д.unique())
    #a.loc[df.ОГРАНКА == 'Р-57','ОГРАНКА']
    df['СРКАРАТ'] = df.КАРАТ/df.ШТУК
    #df.loc[df.СРКАРАТ > 0.3000,'СРКАРАТ'].sort_values().unique()
    print(df.СРКАРАТ.min(),' - ', end = '')#sort_values().unique()
    print(df.СРКАРАТ.max())
    #print(df.loc[a.СРКАРАТ > 2,['id7c','vstavki','vs_dict','gem','ШТУК','ОГРАНКА','КАРАТ','СРКАРАТ','ЦД','ПО','Ц','Д']].sort_values('СРКАРАТ',ascending = False))
    diamonds = df
    diamonds['Col'] = diamonds.Ц.map(dcol_dia)
    diamonds['Cla'] = diamonds.Д.map(dcla_dia)
    diamonds['Po'] = diamonds.ПО.map(dpo_dia)
    diamonds['ogrdisc']= diamonds.ОГРАНКА.map(dogr_dia_discount)
    diamonds['Grp'] = diamonds[['ШТУК','КАРАТ']].apply(lambda x: grp_dia(x.ШТУК,x.КАРАТ), axis=1)
    diamonds['discont_po'] = diamonds.Po.map(discont_po)
    diamonds['ogrbasic'] = diamonds.ОГРАНКА.apply(lambda x: dogr_dia_basic(x))
    res0 = pd.merge(diamonds,PRD[['ogrbasic','Grp','Col','Cla','Pr2019']], how='left',left_on=['ogrbasic','Grp','Col','Cla'], right_on=['ogrbasic','Grp','Col','Cla'])
    res=pd.merge(res0,ogrdiscount[['ogrdisc','Grp','ogrdiscount']], how = 'left', left_on = ['ogrdisc','Grp'], right_on =  ['ogrdisc','Grp'])
    res['prpr'] = round(res.Pr2019*res.ogrdiscount*res.discont_po+0.0000001,0)
    res['prcost'] = round(res.prpr*res.КАРАТ,2)
    print('не сработал прейскурант на этих строках:\n  ', res.loc[pd.isna(res.prpr),['ШТУК',
       'ОГРАНКА', 'КАРАТ', 'ПО', 'Ц', 'Д', 'СРКАРАТ', 'Col',
       'Cla', 'Po', 'ogrdisc', 'Grp', 'discont_po', 'ogrbasic', 'Pr2019',
       'ogrdiscount', 'prpr', 'prcost']])
    return res

In [44]:
#trasform to csv format
carat_to_grp.to_csv("rusprice/CARtoKVG.csv")
KVG57ct.to_csv('rusprice/KVG57ct.csv')
ogrdiscount.to_csv('rusprice/ogrdiscount_f.csv')
PRD.to_csv('rusprice/PR2019_270921.csv')


In [47]:
carat_to_grp.to_csv?

In [50]:
carat_to_grp2 = pd.read_csv("rusprice/CARtoKVG.csv").drop(columns = 'Unnamed: 0')

In [51]:
carat_to_grp2

,grp,caratmin,caratmax
0,400-200,0.00010,0.00499
1,200-120,0.00499,0.00799
2,120-90,0.00799,0.00999
3,90-60,0.00999,0.01499
4,60-40,0.01499,0.02499
...,...,...,...
202,46.00-46.24,46.00000,46.24999
203,46.25-46.49,46.25000,46.49999
204,46.50-46.74,46.50000,46.74999
205,46.75-46.99,46.75000,46.99999


In [39]:
a = pd.read_excel('jewels/prod1C8-111021/vstavki_d_Продажи1с8по11-10-2021manual_only_vstv.xlsx')

a.loc[a.ОГРАНКА == 'Р-57','ОГРАНКА']='Кр-57'

In [53]:
res = rpdiamonds(a)

[]
[]
[]
[]
0.001  - 2.6
не сработал прейскурант на этих строках:
         ШТУК ОГРАНКА  КАРАТ ПО  Ц   Д  СРКАРАТ  Col  Cla Po ogrdisc    Grp  \
1172     1    КР17  0.110  А  2   4    0.110  2.0  4.0  А   Кр-17   10-7   
1356     2    кр57  0.036  Б  3  7А    0.018  3.0  7.5  Б   Кр-57  60-40   

      discont_po ogrbasic  Pr2019  ogrdiscount  prpr  prcost  
1172        1.00    Кр-17     NaN          1.0   NaN     NaN  
1356        0.85    Кр-57     NaN          1.0   NaN     NaN  


In [54]:
res

,Unnamed: 0,Дата док,Магазин,Номер строки,Номенклатура,id7c,Дата док.1,"Номенклатура.Ссылка.Вставки (Справочник ""Номенклатура"" (Общие))",Количество,Вес,...,Cla,Po,ogrdisc,Grp,discont_po,ogrbasic,Pr2019,ogrdiscount,prpr,prcost
0,0,29.04.2021 18:50:25,Благовещенск,4,Кольцо,ЦБ-00019929,29.04.2021 18:50:25,"3Кр57-0,03-3/4А",1,2.56,...,4.0,А,Кр-57,90-60,1.00,Кр-57,445.0,1.0,445.0,13.35
1,1,29.04.2021 18:50:25,Благовещенск,3,Серьги,ЦБ-00051147,29.04.2021 18:50:25,"2Кр57-0,297-3/2А",1,1.34,...,2.0,А,Кр-57,7-6,1.00,Кр-57,800.0,1.0,800.0,237.60
2,2,29.04.2021 18:50:25,Благовещенск,1,Подвеска,ЦБ-00019544,29.04.2021 18:50:25,"1Кр57-0,05-4/7Б",1,1.19,...,7.0,Б,Кр-57,20-15,0.85,Кр-57,280.0,1.0,238.0,11.90
3,3,30.04.2021 17:55:35,Благовещенск,1,Серьги,ЦБ-00021099,30.04.2021 17:55:35,"2Кр57-0,3-3/7А",1,1.34,...,7.0,А,Кр-57,7-6,1.00,Кр-57,405.0,1.0,405.0,121.50
4,4,04.05.2021 18:47:03,Благовещенск,5,Кольцо,ЦБ-00019830,04.05.2021 18:47:03,"1Кр57-0,26-1/3А",1,2.83,...,3.0,А,Кр-57,4-3.4,1.00,Кр-57,1440.0,1.0,1440.0,374.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,1116,30.09.2021 18:45:52,Центральный,3,Серьги,ЦБ-00072914,30.09.2021 18:45:52,"46КР17-0,368-2/4А 4КР57-0,172-3/3А",1,4.83,...,3.0,А,Кр-57,25-20,1.00,Кр-57,425.0,1.0,425.0,73.10
1537,1117,07.10.2021 18:51:01,Центральный,2,Серьги,ЦБ-00029136,07.10.2021 18:51:01,"2Кр57-0,680-5/4А",1,1.62,...,4.0,А,Кр-57,0.30-0.39,1.00,Кр-57,1330.0,1.0,1330.0,904.40
1538,1118,08.10.2021 18:47:30,Центральный,1,Серьги,ЦБ-00029140,08.10.2021 18:47:30,"2Кр57-0,430-4/6А",1,1.43,...,6.0,А,Кр-57,5-4,1.00,Кр-57,525.0,1.0,525.0,225.75
1539,1119,09.10.2021 17:51:20,Центральный,1,Кольцо,ЦБ-00025626,09.10.2021 17:51:20,"1Кр57-0,020-2/3А 2Кр57-0,030-2/3",1,2.52,...,3.0,А,Кр-57,60-40,1.00,Кр-57,490.0,1.0,490.0,9.80


In [43]:
a.columns

Index(['Unnamed: 0', 'Дата док', 'Магазин', 'Номер строки', 'Номенклатура',
       'id7c', 'Дата док.1',
       'Номенклатура.Ссылка.Вставки (Справочник "Номенклатура" (Общие))',
       'Количество', 'Вес', 'Закупочные', 'Розница', 'Выручка', 'Сумма НДС',
       '[% скидки]', 'Прибыль', '[% прибыли]', 'vstavki', 'vs_dict', 'gem',
       'lensourse', 'lendict', 'lenperone', 'ШТУК', 'ОГРАНКА', 'КАРАТ', 'ЦД',
       'ПО', 'Ц', 'Д'],
      dtype='object')